# MRI-Dataset
실제 MRI-Dataset을 활용하기 위하여 <a href="http://adni.loni.usc.edu/">ANDI</a>에서 Dataaset을 다운받았다.  

ANDI에서 제공하는 Dataset은 2가지로 나뉘게 된다.  
Preprocessing이 끝난 Dataset과 Preprocessing이 되지 않은 Dataset을 제공하고 있다.  

먼저 preprocessing이 되지 않은 Dataset을 살펴보면 다음과 같다.(.dcm format)  
![png](./image/6.png)

Dataset의 공통으로 포함되어있는 Description은 다음과 같다.
- B1-Calibration PA
- B1-Calibration Body
- 3-plane localizer
- Axial PD/T2 FSE
- MP-RAGE RESPEAT
- MP-RAGE

이러한 Dataset을 Preprocessing한 Dataset을 살펴보게 되면 다음과 같이 나타내고 있다.(.nii format)  

![png](./image/7.png)
즉, MPR: GradWarp, B1 Correction, N3 Scaled를 활용하여 RAW Dataset을 Quality있는 Dataset으로 Preprocessing하였다는 의미이다.  

이러한 Preprocessing의 의미를 찾아보면 다음과 같다.  
<a href="http://adni.loni.usc.edu/support/experts-knowledge-base/question/?QID=477">ADNI 의 답변</a>을 살펴보게 되면  
1. The “MPR; GradWarp; B1 Correction; N3; Scaled” images come out of the Mayo’s ADIR lab preprocessing pipeline.
2. The “Spatially Normalized, Masked and N3 corrected T1 images” were generated from FreeSurfer, from the UCSF lab who uploaded these processed data.

즉, 위의 Dataset을 **ADIR Lab Preprocessing pipeline or Freesurfer를 이용하여 전처리 과정을 거친다는 것 이다.**  

=> ADNI의 답변과 다른 Site의 물음을 살펴보게 되면, Preprocessing된 File이 Quality가 보장 된 File이라고 칭하고 있다. 여기서 알 수 있는 사실 하나는 **Preprocessing에 사용한 File이 위의 6가지 중 에서 MP-RAGE(MPR) or MP-RAGE RESPECT(MPR-R)중 하나라는 것 이다.**  

읽어보았던 논문을 보면 다음과 같이 언급하고 있다.(A Large Scale Multivariate Parallel ICA Method Reveals Novel Imaging-Genetic Relationships for Alzheimer's Disease in the ADNI Cohort-https://pubmed.ncbi.nlm.nih.gov/22245343/)  
>All subjects underwent a high-resolution 1.5T, 3D structural MRI scan (MPRAGE) as detailed in http://www.adni-info.org. We utilized recently published ADNI imaging data, analyzed in Freesurfer V4.1.0 (http://surfer.nmr.mgh.harvard.edu/fswiki) (Shen et al., 2010), thus, brain structure preparation and analysis methods are described only briefly. An automated Freesurfer Bayesian segmentation and parcellation routine extracted and labeled cortical and subcortical tissue classes (Shen et al., 2010), yielding target region volumes, cortical thicknesses and total intracranial volumes for pre-defined brain structure regions-of-interest (ROIs). Freesurfer values for two inde- pendently collected MPRAGE scans per subject were averaged to yield a single volume/cortical thickness value. Table 1 lists all imaging phenotypes (N = 94; bilateral volumes of interest and cortical thickness values). All values were normalized by Z-score transformation before entry into Para-ICA.

위의 언급에서 주요한 것은 3가지 이다.
1. MRI-Dataset중에서 MPRAGE를 사용하였다.
2. Freesurfer를 사용하여 전처리 결과를 얻었다.
3. Freesurfer의 ROIs로서 값을 나타내어서 값을 얻었다.

즉, MRI-Dataset에서 MPRAGE를 사용(.dcm format)하였고, 이를 Freesurfer를 사용하여 전처리와 ROIs값으로서 나타내었다는 것 이다.  
Quality같은 경우 ANDI가 제공하는 Preprocessing Data또한 대부분 MPR을 사용하였고, 간혹 MPR-R을 사용하였고, FreeSurfer를 사용하였으므로 보장될 것이라고 추측하고 있다.

# Preprocessing ADNI MRI Dataset
목표로 하는 Preprocessing Pipeline은 다음과 같다.
1. 개개인의 <a href="http://adni.loni.usc.edu/">ADNI Dataset</a> 중 Quality가 높은 MP-RAGE(166개의 Slice)를 다운로드 받는다.
2. 166개의 .dcm file을 하나의 .nii.gz로서 변환한다. 
 - .dcm file: MRI 사진의 한 Slice로서 환자에 대한 Information과 사진을 가지고 있다.
 - .nii file: MRI의 Slice를 모두 하나로 합쳐 3D Image로 만듬과 동시에 환자의 Infromation도 가지고 있는 File Format이다.
3. Freesurfer를 통하여 Preprocessing(recon-all)을 실시한다. 
4. Freesurfer의 결과중 관심있는 특정 ROI Extraction한다.
5. Extraction한 Feature를 Age, Sex Correction을 실시하게 된다.

**참조**  
1. 1의 과정은 중요한 것이 아니라, 여러 CSV File을 읽고 Mapping하는 단계여서 제외하였습니다.  
2. <span style="color:red;">현재 1~4에서 다루는 Data의 크기는 대략적으로 150G입니다. 따라서, GitHub에 올리지 못하는 상황입니다. 따라서 GitHub에 올려서 Code로서 실행하는 구간은 4의 결과의 Data로부터 시작합니다.</span>
3. 3의 과정은 대략, 4주 정도 걸리는 전처리 과정이였으나 Project Proposal 발표 때, 3의 과정이 거의 끝나가는 시점에서 발표하였습니다.

## 2. Convert .dcm to .nii.gz File
특정 환자의 여러개의 .dcm File을 -> .nii.gz File로서 Convert를 하는 방법 이다.

.dcm File을 .nii.gz File로서 Convert하기 위하여 dcm2niix(https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage)를 사용하였다. GUI를 지원하여 Directory별로 쉽게 접근 가능하여 선택하였던 Tool이다. 대략 한 Subject당 1s이하로 작업을 마칠 수 있었다. (GUI같은 경우는 MRIcroGL(https://www.nitrc.org/projects/mricrogl/)이 필요하다.)

## 3. Freesurfer recon-all Parrel
하나의 .nii.gz에 대하여 6~24시간이 걸리게 되고, 수행되는 과정은 다음과 같다.(https://surfer.nmr.mgh.harvard.edu/fswiki/recon-all)  

**위의 Tool은 Linux상에서 존재하는 parallel을 지원하며, 병렬적으로 수행할 수 있었다.**  

Server상에서 옮겨 CPU 16개를 사용하는 명령어는 다음과 같다.
```code
export SUBJECT_DIR=./
echo $SUBJECT_DIR
ls *.nii.gz | parallel --jobs 16 recon-all -s {.} -i {} -all -qcache
```

## 4. FreeSurfer ROI Extraction Command
**참고**: 현재 Freesurfer ROI는 Python 2.x에서 작동합니다. Python가상환경으로 2.x를 만들어서 실행하셔야 돌아갑니다.

```code
subj_list_txt=$(<subject_list.txt)
output_folder=ROI

### gray matter

**area**  
measure=area

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt


**thickness**  
measure=thickness

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt

**thicknessstd**  
measure=thicknessstd

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt

 
**meancurv**  
measure=meancurv

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt

**gauscurv**  
measure=gauscurv

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt

 

**foldind**  
measure=foldind

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt

 

**curvind**  
measure=curvind

aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas ${measure} --tablefile ${output_folder}/${measure}_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas ${measure} --tablefile ${output_folder}/${measure}_lh.txt

 

**volume**  
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi rh --meas volume --tablefile ${output_folder}/cortical_volume_rh.txt
aparcstats2table --subjects ${subj_list_txt} --parc aparc --hemi lh --meas volume --tablefile ${output_folder}/cortical_volume_lh.txt

 

### subcortical volume
asegstats2table --subjects ${subj_list_txt} --meas volume --tablefile ${output_folder}/subcortical_volume.txt

 

### white matter volume
asegstats2table --subjects ${subj_list_txt} --stats wmparc.stats --tablefile ${output_folder}/white_matter_volume.txt
```

위와 같은 과정을 거치기 전에 문제가 있는 Subject를 제외하고 ROI Extraction을 실행하게 된다.  
먼저 Sex-Check에서 Error가 발생한 Subject는 다음과 같이 2명이다.  
또한 Freesurfer에서 처리를 못한 Subject의 경우 다음과 같다.  
- Plink: Subject 073_S_0909, 130_S_1201 (SNPs Data Error)
- Freesurfer: 005_S_1224, 018_S_0043, 100_S_0747

즉, 총 5명의 Subject가 제외된 752명으로서 진행되게 된다.

위에서 추출한 Feature Extraction에서 주요하지 않은 Feature는 삭제하고 관심있는 Feature를 합치는 방법입니다.  
각각의 사용한 Feature는 다음과 같습니다.
- Area
- Thickness
- Cortical Volume
- Sub Cortical Volume
- White Matter Volume

### Area

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

file_path = "./MRI_DATA/"

In [2]:
# Area
area_rh = pd.read_csv(file_path+"area_rh.txt",sep='\t')
area_lh = pd.read_csv(file_path+"area_lh.txt",sep='\t')

In [3]:
area_rh.head()

,rh.aparc.area,rh_bankssts_area,rh_caudalanteriorcingulate_area,rh_caudalmiddlefrontal_area,rh_cuneus_area,rh_entorhinal_area,rh_fusiform_area,rh_inferiorparietal_area,rh_inferiortemporal_area,rh_isthmuscingulate_area,...,rh_superiorparietal_area,rh_superiortemporal_area,rh_supramarginal_area,rh_frontalpole_area,rh_temporalpole_area,rh_transversetemporal_area,rh_insula_area,rh_WhiteSurfArea_area,BrainSegVolNotVent,eTIV
0,128_S_0225.nii,701.0,436.0,1828.0,1481.0,389.0,2519.0,4321.0,2571.0,832.0,...,5032.0,3179.0,3169.0,321.0,430.0,310.0,2018.0,71299.3,869726.0,1.330440e+06
1,021_S_0273.nii,933.0,757.0,2077.0,1844.0,344.0,2882.0,6356.0,3162.0,1204.0,...,6348.0,3847.0,3497.0,335.0,460.0,353.0,2547.0,90216.1,1109633.0,1.766893e+06
2,029_S_1384.nii,642.0,971.0,2392.0,2085.0,479.0,3226.0,5424.0,3311.0,1036.0,...,6550.0,3095.0,3253.0,342.0,520.0,272.0,2136.0,92657.3,1097353.0,1.713025e+06
3,014_S_0169.nii,778.0,728.0,2019.0,1749.0,388.0,3237.0,5278.0,3651.0,1134.0,...,4614.0,3628.0,3185.0,299.0,550.0,297.0,2959.0,87045.1,1069012.0,1.531117e+06
4,141_S_1052.nii,1062.0,597.0,2052.0,1417.0,292.0,2752.0,6369.0,3216.0,861.0,...,4808.0,3435.0,2995.0,289.0,390.0,311.0,2316.0,81368.2,994316.0,1.452094e+06


In [4]:
area_rh = area_rh.drop(['BrainSegVolNotVent','eTIV'],axis=1)
area_lh = area_lh.drop(['BrainSegVolNotVent','eTIV'],axis=1)

In [5]:
# Subject, BrainSegVolNotVent, eTIV
area_rh.rename(columns={"rh.aparc.area":"Subject"},inplace=True)
area_lh.rename(columns={"lh.aparc.area":"Subject"},inplace=True)

num_of_fearue_area = len(area_rh.iloc[0])+len(area_lh.iloc[0])-2

### Thickness

In [6]:
# Thickness
thickness_rh = pd.read_csv(file_path+"thickness_rh.txt",sep='\t')
thickness_lh = pd.read_csv(file_path+"thickness_lh.txt",sep='\t')

thickness_rh.head()

,rh.aparc.thickness,rh_bankssts_thickness,rh_caudalanteriorcingulate_thickness,rh_caudalmiddlefrontal_thickness,rh_cuneus_thickness,rh_entorhinal_thickness,rh_fusiform_thickness,rh_inferiorparietal_thickness,rh_inferiortemporal_thickness,rh_isthmuscingulate_thickness,...,rh_superiorparietal_thickness,rh_superiortemporal_thickness,rh_supramarginal_thickness,rh_frontalpole_thickness,rh_temporalpole_thickness,rh_transversetemporal_thickness,rh_insula_thickness,rh_MeanThickness_thickness,BrainSegVolNotVent,eTIV
0,128_S_0225.nii,2.208,2.790,2.321,1.902,3.497,2.515,2.271,2.658,2.200,...,1.909,2.423,2.198,2.765,3.542,2.020,2.872,2.26325,869726.0,1.330440e+06
1,021_S_0273.nii,2.318,2.240,2.258,1.699,3.692,2.722,2.126,2.616,2.329,...,1.801,2.608,2.356,2.172,3.562,2.127,3.006,2.27758,1109633.0,1.766893e+06
2,029_S_1384.nii,2.147,2.254,1.863,1.674,2.430,2.300,2.068,2.291,2.170,...,1.681,2.363,2.115,2.146,3.527,2.470,2.908,2.03832,1097353.0,1.713025e+06
3,014_S_0169.nii,2.131,2.352,1.979,1.619,3.452,2.367,2.011,2.441,1.915,...,1.746,2.470,2.071,1.840,3.642,1.841,2.798,2.07758,1069012.0,1.531117e+06
4,141_S_1052.nii,2.555,2.232,2.288,1.723,3.451,2.624,2.133,2.718,2.050,...,1.802,2.577,2.378,2.566,3.797,2.341,2.784,2.31495,994316.0,1.452094e+06


In [7]:
thickness_rh = thickness_rh.drop(['rh_MeanThickness_thickness','BrainSegVolNotVent','eTIV'],axis=1)
thickness_lh = thickness_lh.drop(['lh_MeanThickness_thickness','BrainSegVolNotVent','eTIV'],axis=1)

In [8]:
# Subject, MeanThickness_thckness, BrainSegVolNotVent, eTIV
thickness_rh.rename(columns={"rh.aparc.thickness":"Subject"},inplace=True)
thickness_lh.rename(columns={"lh.aparc.thickness":"Subject"},inplace=True)

num_of_fearue_thickness = len(thickness_rh.iloc[0])+len(thickness_lh.iloc[0])-2

### Cortical Volume

In [9]:
# Cortical Volume
cortical_volume_rh = pd.read_csv(file_path+"cortical_volume_rh.txt",sep='\t')
cortical_volume_lh = pd.read_csv(file_path+"cortical_volume_lh.txt",sep='\t')

cortical_volume_rh.head()

,rh.aparc.volume,rh_bankssts_volume,rh_caudalanteriorcingulate_volume,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,rh_isthmuscingulate_volume,...,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume,BrainSegVolNotVent,eTIV
0,128_S_0225.nii,1516.0,1178.0,4662.0,2894.0,1771.0,7435.0,10821.0,8000.0,2315.0,...,15912.0,10273.0,8919.0,7702.0,1110.0,2093.0,741.0,5994.0,869726.0,1.330440e+06
1,021_S_0273.nii,2069.0,1984.0,5039.0,3289.0,1832.0,9028.0,15446.0,9981.0,3067.0,...,19057.0,12246.0,11103.0,8850.0,922.0,2482.0,799.0,7516.0,1109633.0,1.766893e+06
2,029_S_1384.nii,1254.0,2401.0,4396.0,3592.0,1696.0,8771.0,11718.0,8443.0,2355.0,...,16660.0,11040.0,8384.0,7111.0,823.0,2691.0,758.0,6726.0,1097353.0,1.713025e+06
3,014_S_0169.nii,1531.0,1875.0,4105.0,2907.0,1964.0,7982.0,11016.0,9725.0,2367.0,...,17330.0,8210.0,9761.0,6742.0,703.0,3063.0,495.0,8464.0,1069012.0,1.531117e+06
4,141_S_1052.nii,2636.0,1397.0,5181.0,2691.0,1572.0,7941.0,14878.0,10507.0,1955.0,...,20959.0,9230.0,9883.0,7773.0,982.0,2295.0,752.0,6461.0,994316.0,1.452094e+06


In [10]:
cortical_volume_rh = cortical_volume_rh.drop(['BrainSegVolNotVent','eTIV'],axis=1)
cortical_volume_lh = cortical_volume_lh.drop(['BrainSegVolNotVent','eTIV'],axis=1)

In [11]:
# Subject, BrainSegVolNotVent, eTIV
cortical_volume_rh.rename(columns={"rh.aparc.volume":"Subject"},inplace=True)
cortical_volume_lh.rename(columns={"lh.aparc.volume":"Subject"},inplace=True)

num_of_fearue_cortical_volume = len(cortical_volume_rh.iloc[0])+len(cortical_volume_lh.iloc[0])-2

### Sub Cortical Volume

In [12]:
# Sub Cortical Volume
sub_cortical_volume = pd.read_csv(file_path+"subcortical_volume.txt",sep='\t')

sub_cortical_volume.head()

,Measure:volume,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,3rd-Ventricle,...,SupraTentorialVol,SupraTentorialVolNotVent,SupraTentorialVolNotVentVox,MaskVol,BrainSegVol-to-eTIV,MaskVol-to-eTIV,lhSurfaceHoles,rhSurfaceHoles,SurfaceHoles,EstimatedTotalIntraCranialVol
0,128_S_0225.nii,17356.4,1073.5,10048.7,43045.1,5474.1,2551.7,3650.1,1296.6,1850.3,...,8.012646e+05,765245.635816,763242.0,1250152.0,0.683859,0.939653,28.0,13.0,41.0,1.330440e+06
1,021_S_0273.nii,21933.8,645.2,13058.5,46999.1,6854.3,3114.7,4114.9,1437.2,1189.0,...,1.040986e+06,994027.049714,992655.0,1663328.0,0.656718,0.941386,73.0,57.0,130.0,1.766893e+06
2,029_S_1384.nii,29480.4,799.7,11983.1,48396.9,6475.4,2776.2,3828.2,1648.3,1875.5,...,1.044279e+06,977478.515955,972677.0,1715157.0,0.682372,1.001244,49.0,39.0,88.0,1.713025e+06
3,014_S_0169.nii,19566.6,542.8,11190.9,46168.7,6817.6,3053.7,4040.2,1786.8,2658.2,...,9.960726e+05,954706.588064,953225.0,1539532.0,0.729853,1.005496,23.0,18.0,41.0,1.531117e+06
4,141_S_1052.nii,16128.5,262.4,11730.4,47837.2,5829.0,3332.2,4106.5,1665.9,1558.4,...,9.108869e+05,878615.859361,876044.0,1461628.0,0.710359,1.006566,30.0,20.0,50.0,1.452094e+06


In [13]:
sub_cortical_volume = sub_cortical_volume.drop(['EstimatedTotalIntraCranialVol','SurfaceHoles','rhSurfaceHoles','lhSurfaceHoles','MaskVol-to-eTIV','BrainSegVol-to-eTIV','MaskVol'],axis=1)
sub_cortical_volume = sub_cortical_volume.drop(['Right-non-WM-hypointensities','Left-non-WM-hypointensities','non-WM-hypointensities','Right-WM-hypointensities','Left-WM-hypointensities'],axis=1)

In [14]:
# Subject + Lots of Features
sub_cortical_volume.rename(columns={"Measure:volume":"Subject"},inplace=True)

num_of_fearue_sub_cortical_volume = len(sub_cortical_volume.iloc[0])-1

### White Matter Volume

In [15]:
# White Matter Volume
white_matter_volume = pd.read_csv(file_path+"white_matter_volume.txt",sep='\t')

white_matter_volume.head()

,Measure:volume,wm-lh-bankssts,wm-lh-caudalanteriorcingulate,wm-lh-caudalmiddlefrontal,wm-lh-cuneus,wm-lh-entorhinal,wm-lh-fusiform,wm-lh-inferiorparietal,wm-lh-inferiortemporal,wm-lh-isthmuscingulate,...,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter,lhCerebralWhiteMatterVol,rhCerebralWhiteMatterVol,CerebralWhiteMatterVol,MaskVol,EstimatedTotalIntraCranialVol
0,128_S_0225.nii,1623.4,1754.4,5581.5,2264.0,857.4,5340.7,6843.0,4348.1,3365.0,...,631.2,492.6,8546.7,17371.8,16647.3,180621.636957,180075.794833,360697.431791,1250152.0,1.330440e+06
1,021_S_0273.nii,3336.9,2546.9,7104.4,3162.8,974.7,7208.9,11467.8,6616.2,4255.5,...,704.7,822.7,10389.1,32091.7,35014.6,243547.654799,245234.607168,488782.261967,1663328.0,1.766893e+06
2,029_S_1384.nii,2265.3,2813.0,9058.5,3850.2,1135.0,7152.8,9996.4,5643.6,4240.4,...,831.0,518.7,9616.9,31495.4,31276.8,264560.214543,260467.368918,525027.583461,1715157.0,1.713025e+06
3,014_S_0169.nii,3804.0,2528.4,6931.6,2869.5,871.9,7826.4,12263.1,8788.3,4218.1,...,855.7,808.9,11585.9,33880.5,33467.1,263984.943841,259565.449616,523550.393458,1539532.0,1.531117e+06
4,141_S_1052.nii,2233.1,2765.4,5789.0,2091.3,966.2,5114.2,10374.6,6581.4,3534.6,...,588.9,789.2,9446.7,19570.0,19837.7,204559.156746,205532.184865,410091.341610,1461628.0,1.452094e+06


In [16]:
white_matter_volume = white_matter_volume.drop(['MaskVol','EstimatedTotalIntraCranialVol'],axis=1)

In [17]:
# Subject, MaskVol, EstimatedTotalIntraCranialVol
white_matter_volume.rename(columns={"Measure:volume":"Subject"},inplace=True)

num_of_fearue_white_matter_volume = len(white_matter_volume.iloc[0])-1

In [18]:
# Total Feature of Collective Deep Matrix
print('Total Feature of Collective Deep Matrix: ',num_of_fearue_area+num_of_fearue_cortical_volume+num_of_fearue_sub_cortical_volume+num_of_fearue_thickness+num_of_fearue_white_matter_volume)

Total Feature of Collective Deep Matrix:  333


### Merge All of ROI Features

In [19]:
CDMF_area = pd.merge(area_rh,area_lh,on='Subject')
CDMF_thickness = pd.merge(thickness_rh,thickness_lh,on='Subject')
CDMF_Volume = pd.merge(cortical_volume_rh,cortical_volume_lh,on='Subject')
CDMF_others = pd.merge(sub_cortical_volume,white_matter_volume, on='Subject')

CDMF_1 = pd.merge(CDMF_area,CDMF_thickness,on='Subject')
CDMF_2 = pd.merge(CDMF_Volume,CDMF_others,on='Subject')

CDMF_ROI = pd.merge(CDMF_1,CDMF_2,on='Subject')

CDMF_ROI

,Subject,rh_bankssts_area,rh_caudalanteriorcingulate_area,rh_caudalmiddlefrontal_area,rh_cuneus_area,rh_entorhinal_area,rh_fusiform_area,rh_inferiorparietal_area,rh_inferiortemporal_area,rh_isthmuscingulate_area,...,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter,lhCerebralWhiteMatterVol_y,rhCerebralWhiteMatterVol_y,CerebralWhiteMatterVol_y
0,128_S_0225.nii,701.0,436.0,1828.0,1481.0,389.0,2519.0,4321.0,2571.0,832.0,...,7749.0,423.0,631.2,492.6,8546.7,17371.8,16647.3,180621.636957,180075.794833,360697.431791
1,021_S_0273.nii,933.0,757.0,2077.0,1844.0,344.0,2882.0,6356.0,3162.0,1204.0,...,8909.3,338.1,704.7,822.7,10389.1,32091.7,35014.6,243547.654799,245234.607168,488782.261967
2,029_S_1384.nii,642.0,971.0,2392.0,2085.0,479.0,3226.0,5424.0,3311.0,1036.0,...,7830.8,503.5,831.0,518.7,9616.9,31495.4,31276.8,264560.214543,260467.368918,525027.583461
3,014_S_0169.nii,778.0,728.0,2019.0,1749.0,388.0,3237.0,5278.0,3651.0,1134.0,...,8383.7,372.6,855.7,808.9,11585.9,33880.5,33467.1,263984.943841,259565.449616,523550.393458
4,141_S_1052.nii,1062.0,597.0,2052.0,1417.0,292.0,2752.0,6369.0,3216.0,861.0,...,6695.0,302.7,588.9,789.2,9446.7,19570.0,19837.7,204559.156746,205532.184865,410091.341610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,123_S_0072.nii,840.0,881.0,1703.0,1710.0,389.0,2709.0,5891.0,3656.0,924.0,...,7611.8,356.3,842.3,697.5,9027.1,30417.2,31855.4,230730.688478,229958.832526,460689.521003
748,136_S_0579.nii,979.0,533.0,2061.0,1416.0,454.0,2759.0,4880.0,3231.0,825.0,...,9226.0,392.0,758.1,660.7,9896.6,38203.4,38301.8,251143.217295,252211.813038,503355.030333
749,067_S_0098.nii,547.0,605.0,1646.0,1671.0,332.0,2033.0,3534.0,2012.0,795.0,...,7020.0,254.2,673.1,666.8,9057.6,19531.4,21191.3,186873.704686,188851.958189,375725.662875
750,027_S_0835.nii,1072.0,719.0,2308.0,1465.0,323.0,2825.0,5674.0,3218.0,894.0,...,10839.5,441.2,559.7,709.0,10850.3,39353.6,37696.4,280253.091285,274836.116993,555089.208278


In [20]:
# Change Subject_Name
CDMF_ROI['Subject'] = CDMF_ROI['Subject'].apply(lambda x: x.split('.')[0])

# Save
CDMF_ROI.to_csv(file_path+'CDMF_ROI.csv')
# Load
CDMF_ROI = pd.read_csv(file_path+'CDMF_ROI.csv',index_col=0)

CDMF_ROI.head()

,Subject,rh_bankssts_area,rh_caudalanteriorcingulate_area,rh_caudalmiddlefrontal_area,rh_cuneus_area,rh_entorhinal_area,rh_fusiform_area,rh_inferiorparietal_area,rh_inferiortemporal_area,rh_isthmuscingulate_area,...,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter,lhCerebralWhiteMatterVol_y,rhCerebralWhiteMatterVol_y,CerebralWhiteMatterVol_y
0,128_S_0225,701.0,436.0,1828.0,1481.0,389.0,2519.0,4321.0,2571.0,832.0,...,7749.0,423.0,631.2,492.6,8546.7,17371.8,16647.3,180621.636957,180075.794833,360697.431791
1,021_S_0273,933.0,757.0,2077.0,1844.0,344.0,2882.0,6356.0,3162.0,1204.0,...,8909.3,338.1,704.7,822.7,10389.1,32091.7,35014.6,243547.654799,245234.607168,488782.261967
2,029_S_1384,642.0,971.0,2392.0,2085.0,479.0,3226.0,5424.0,3311.0,1036.0,...,7830.8,503.5,831.0,518.7,9616.9,31495.4,31276.8,264560.214543,260467.368918,525027.583461
3,014_S_0169,778.0,728.0,2019.0,1749.0,388.0,3237.0,5278.0,3651.0,1134.0,...,8383.7,372.6,855.7,808.9,11585.9,33880.5,33467.1,263984.943841,259565.449616,523550.393458
4,141_S_1052,1062.0,597.0,2052.0,1417.0,292.0,2752.0,6369.0,3216.0,861.0,...,6695.0,302.7,588.9,789.2,9446.7,19570.0,19837.7,204559.156746,205532.184865,410091.341610


## 5. Age, Sex Correction

<p>$$X^{obs} = w_1 \cdot age + w_2 \cdot sex + w_3 \cdot cohort + X^{ori}$$</p>
현재 얻을 ADNI1의 Data만 사용하므로 cohort는 제외되어서 최종적인 식은 다음과 같이 정의된다.
<p>$$X^{obs} = w_1 \cdot age + w_2 \cdot sex+X^{ori}$$</p>

Trainning  
<p>$$w^{*} = min_w \sum_{i=1}^{n} (x^t t_i-X_i^{obs})^2$$</p>

**1. Z-Normalization**

In [21]:
CDMF_ROI = pd.read_csv(file_path+'CDMF_ROI.csv',index_col=0)

# 1. Z_Normlaizaiton
for j,a in enumerate(CDMF_ROI.columns[1:]):
    CDMF_ROI[CDMF_ROI.columns[j+1]] = (CDMF_ROI[CDMF_ROI.columns[j+1]]-CDMF_ROI[CDMF_ROI.columns[j+1]].mean())/CDMF_ROI[CDMF_ROI.columns[j+1]].std()

**2. Load Subject Age, Sex Information**

In [23]:
visit_screening = pd.read_csv(file_path+'Visit_Screening.csv')
visit_screening.head()

,Subject ID,Sex,Age,Description
0,002_S_0295,M,84.9,B1-Calibration Body
1,002_S_0295,M,84.9,B1-Calibration PA
2,002_S_0295,M,84.9,3-plane localizer
3,002_S_0295,M,84.9,3-plane localizer
4,002_S_0295,M,84.9,3-plane localizer


**3. Add Age, Sex Information**

In [24]:
CDMF_ROI["sex"]=0
CDMF_ROI["age"]=0

for s in CDMF_ROI["Subject"]:
    try:
        sex = visit_screening[visit_screening['Subject ID']==s].Sex.unique()[0]
        age = visit_screening[visit_screening['Subject ID']==s].Age.unique()[0]
        
        if sex == 'M':
            sex = 1
        else:
            sex = -1
            
        CDMF_ROI.loc[CDMF_ROI["Subject"] == s,'age'] = age
        CDMF_ROI.loc[CDMF_ROI["Subject"] == s,'sex'] = sex
    except:
        print("Subject Error: ",s)
        
CDMF_ROI.head()

Subject Error:  009_S_1030a


,Subject,rh_bankssts_area,rh_caudalanteriorcingulate_area,rh_caudalmiddlefrontal_area,rh_cuneus_area,rh_entorhinal_area,rh_fusiform_area,rh_inferiorparietal_area,rh_inferiortemporal_area,rh_isthmuscingulate_area,...,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter,lhCerebralWhiteMatterVol_y,rhCerebralWhiteMatterVol_y,CerebralWhiteMatterVol_y,sex,age
0,128_S_0225,-0.917548,-1.664036,-0.596404,-0.221529,-0.092980,-0.841548,-0.729943,-0.868566,-0.289421,...,-0.395086,-1.126002,-1.003506,-1.713866,-1.867349,-1.584602,-1.608230,-1.602320,-1,82.4
1,021_S_0273,1.005094,0.611490,0.152804,1.370009,-0.682973,0.174082,2.083932,0.436097,2.293321,...,0.125246,1.101439,0.411719,0.536352,0.962954,0.508389,0.562182,0.537249,1,62.9
2,029_S_1384,-1.406496,2.128508,1.100598,2.426650,1.087007,1.136553,0.795219,0.765023,1.126921,...,1.019369,-0.949885,-0.181440,0.445196,0.386979,1.207291,1.069579,1.142702,1,72.1
3,014_S_0169,-0.279430,0.405913,-0.021710,0.953491,-0.106091,1.167330,0.593339,1.515590,1.807321,...,1.194229,1.008319,1.331032,0.809804,0.724492,1.188157,1.039536,1.118026,-1,72.1
4,141_S_1052,2.074149,-0.522729,0.077582,-0.502131,-1.364744,-0.189642,2.101907,0.555305,-0.088078,...,-0.694543,0.875388,-0.312178,-1.377829,-1.375726,-0.788413,-0.760288,-0.777229,-1,69.1


**3. Quality Control**  
sklearn의 Linear Regression Model에 넣기 전에 Data의 Quality를 검사한다.

In [25]:
print('Num of Nan: ',CDMF_ROI.isna().sum().sum())

Num of Nan:  0


**4. Linear Regression Fitting**

1. Linear Regression Model Fitting: $w^{*} = min_w \sum_{i=1}^{n} (x^t t_i-X_i^{obs})^2$
2. Apply Model in CDMF ROI: $X^{ori} =  X^{obs} - w_1 \cdot age + w_2 \cdot sex$

In [26]:
# 1. Linear Regression Fitting
ROI_copy = CDMF_ROI.copy()

sex = ROI_copy.pop('sex')
age = ROI_copy.pop('age')

subjects = ROI_copy.pop('Subject')

label = ROI_copy
dataset = pd.concat([sex,age],axis=1)

line_fitter = LinearRegression()
line_fitter.fit(dataset,label)

# 2. Apply Model in CDMF ROI
ROI_image_preprocessing = CDMF_ROI.copy()
col = ROI_image_preprocessing.columns[1:-2]

for i in range(len(ROI_image_preprocessing)):
    age = ROI_image_preprocessing.iloc[i].age
    sex = ROI_image_preprocessing.iloc[i].sex
    
    predict = line_fitter.predict([[sex,age]])
    
    for j,c in enumerate(col):
        ROI_image_preprocessing.iloc[i,j+1] = ROI_image_preprocessing.iloc[i,j+1] - predict[0,j]
        
# 4. Check the Result
ROI_image_preprocessing.to_csv(file_path+'CDMF_ROI_Preprocessing.csv')
ROI_preprocess = pd.read_csv(file_path+'CDMF_ROI_Preprocessing.csv',index_col=0)
ROI_preprocess

,Subject,rh_bankssts_area,rh_caudalanteriorcingulate_area,rh_caudalmiddlefrontal_area,rh_cuneus_area,rh_entorhinal_area,rh_fusiform_area,rh_inferiorparietal_area,rh_inferiortemporal_area,rh_isthmuscingulate_area,...,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter,lhCerebralWhiteMatterVol_y,rhCerebralWhiteMatterVol_y,CerebralWhiteMatterVol_y,sex,age
0,128_S_0225,-0.434351,-1.210666,-0.152256,0.434245,0.375762,-0.053796,-0.114782,-0.130181,0.222878,...,0.265007,-0.848383,-0.308552,-1.048192,-1.192761,-0.751647,-0.784488,-0.770900,-1,82.4
1,021_S_0273,0.551874,0.195456,-0.240765,0.766726,-1.058745,-0.560424,1.567169,-0.258386,1.866972,...,-0.496226,0.887284,-0.227797,-0.233391,0.189877,-0.314974,-0.249744,-0.283428,1,62.9
2,029_S_1384,-1.708161,1.843119,0.817446,2.014236,0.778091,0.643638,0.395823,0.304528,0.793288,...,0.607860,-1.134985,-0.618447,0.066643,0.001579,0.697990,0.565318,0.634042,1,72.1
3,014_S_0169,0.034091,0.713017,0.298818,1.395575,0.287803,1.684606,1.077099,1.992010,2.215820,...,1.619255,1.253409,1.799264,1.037515,0.965050,1.669498,1.518828,1.600122,-1,72.1
4,141_S_1052,2.338250,-0.258227,0.362105,-0.122287,-0.992651,0.248854,2.547396,0.955424,0.290187,...,-0.337984,1.111004,0.090018,-1.277680,-1.261584,-0.409483,-0.381322,-0.396877,-1,69.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,123_S_0072,-0.090348,1.185240,-1.272467,0.341040,-0.412070,-0.839630,1.023702,1.030526,0.001580,...,0.655906,0.067096,-1.102314,-0.157709,0.031744,-0.475010,-0.497727,-0.488162,1,70.7
748,136_S_0579,1.591104,-0.763036,0.345978,-0.201359,1.105165,0.173904,0.442571,0.896977,0.003964,...,0.777684,0.232546,0.356363,1.417717,1.431936,1.017062,1.053166,1.038934,-1,65.5
749,067_S_0098,-1.675994,0.017173,-0.674666,1.310850,-0.356302,-1.358420,-1.176208,-1.310793,-0.012844,...,0.609558,0.333711,0.130117,-0.628763,-0.404064,-0.472008,-0.421930,-0.448642,-1,84.5
750,027_S_0835,1.868535,0.068085,0.574303,-0.687496,-1.261407,-0.457304,0.751714,0.119572,-0.184535,...,-1.294511,0.151643,0.346589,1.301936,1.024517,1.247263,1.070688,1.163332,1,72.9


In [27]:
ROI_preprocess.to_csv(file_path+'CDMF_ROI_Final.csv')
ROI_preprocess = pd.read_csv(file_path+'CDMF_ROI_Final.csv',index_col=0)